In [2]:
import csv

def get_int(prompt):
    while True:
        try:
            value = int(input(prompt))
            return value
        except ValueError:
            print("Error: Ingrese un valor entero.")

def get_float(prompt):
    while True:
        try:
            value = float(input(prompt))
            return value
        except ValueError:
            print("Error: Ingrese un valor numérico (Integer o Float).")

def get_uppercase(prompt):
    while True:
        value = input(prompt).strip()
        if value.isalpha() and value.upper() == value:
            return value
        else:
            print("Error: Ingrese solo letras mayúsculas.")

def main():
    # Lista donde se almacenarán las filas para el CSV
    filas = []
    
    # Diccionario para llevar el conteo de hijos asignados a cada nodo predecesor
    hijos_contador = {}
    # Diccionario para guardar el último "j" (segundo elemento del par) de cada nodo
    ultimo_j = {}
    
    # Inicializamos el nodo INICIO
    nodo_inicio = {
        "Nodo": "INICIO",
        "Actividad": "INICIO",
        "Predecesor": "",
        "Tiempo de actividad": 0,
        "Par ordenado": "(0,0)"
    }
    filas.append(nodo_inicio)
    hijos_contador["INICIO"] = 0
    ultimo_j["INICIO"] = 0

    # Solicitamos el número total de actividades (nodos intermedios)
    num_actividades = get_int("Ingrese el número de actividades (nodos): ")

    # Variable para recordar el nombre del último nodo ingresado (para FIN)
    ultimo_nodo = None

    # Procesamos cada actividad
    for idx in range(num_actividades):
        print(f"\n--- Ingresando datos para la actividad {idx+1} ---")
        nodo = get_uppercase("Ingrese el nombre del nodo (solo letras mayúsculas): ")
        actividad = input("Ingrese el nombre de la actividad: ").strip()
        ultimo_nodo = nodo  # Actualizamos el último nodo ingresado

        num_pred = get_int("Ingrese el número de nodos predecesores (0 significa que depende de INICIO): ")
        
        # Caso: Si se ingresa 0, se asigna INICIO como predecesor y se pide la duración de la actividad.
        if num_pred == 0:
            pred = "INICIO"
            dur = get_float("Ingrese la duración de la actividad: ")
            total_rel = 1  # Se trata como una relación única
            
            # Obtenemos el "j" del predecesor
            base = ultimo_j[pred]
            # Número de hijos ya asignados para este predecesor
            cont = hijos_contador.get(pred, 0)
            # Para una única relación se usa incremento 1
            inc = 1
            nuevo_j = base + (cont + 1) * inc
            par = (base, nuevo_j)
            hijos_contador[pred] = cont + 1
            # Actualizamos el último "j" para el nodo actual
            ultimo_j[nodo] = nuevo_j

            filas.append({
                "Nodo": nodo,
                "Actividad": actividad,
                "Predecesor": pred,
                "Tiempo de actividad": dur,
                "Par ordenado": f"({par[0]},{par[1]})"
            })
        
        # Caso: Si se ingresa 1, se pide el nombre del predecesor y la duración de la actividad.
        elif num_pred == 1:
            pred = get_uppercase("Ingrese el nombre del nodo predecesor: ")
            dur = get_float("Ingrese la duración de la actividad: ")
            total_rel = 1

            if pred not in ultimo_j:
                print(f"Advertencia: El nodo predecesor {pred} no ha sido definido. Se usará base 0.")
                base = 0
                ultimo_j[pred] = 0
                hijos_contador[pred] = 0
            else:
                base = ultimo_j[pred]
            cont = hijos_contador.get(pred, 0)
            # Para única relación se usa incremento 1
            inc = 1
            nuevo_j = base + (cont + 1) * inc
            par = (base, nuevo_j)
            hijos_contador[pred] = cont + 1
            ultimo_j[nodo] = nuevo_j

            filas.append({
                "Nodo": nodo,
                "Actividad": actividad,
                "Predecesor": pred,
                "Tiempo de actividad": dur,
                "Par ordenado": f"({par[0]},{par[1]})"
            })
        
        # Caso: Si se ingresa 2 o más, se pregunta para cada relación el nombre del predecesor y la duración.
        elif num_pred >= 2:
            # Aquí usaremos incremento 2 para cada relación de la actividad
            for i in range(num_pred):
                pred = get_uppercase(f"Ingrese el nombre del nodo predecesor {i+1}: ")
                dur = get_float(f"Ingrese la duración de la actividad del predecesor {pred}: ")
                if pred not in ultimo_j:
                    print(f"Advertencia: El nodo predecesor {pred} no ha sido definido. Se usará base 0.")
                    base = 0
                    ultimo_j[pred] = 0
                    hijos_contador[pred] = 0
                else:
                    base = ultimo_j[pred]
                cont = hijos_contador.get(pred, 0)
                inc = 2  # Para múltiples relaciones se usa incremento 2
                nuevo_j = base + (cont + 1) * inc
                par = (base, nuevo_j)
                hijos_contador[pred] = cont + 1
                # Si la actividad ya se ingresó con otra relación, se guarda el mayor "j"
                if nodo in ultimo_j:
                    ultimo_j[nodo] = max(ultimo_j[nodo], nuevo_j)
                else:
                    ultimo_j[nodo] = nuevo_j

                filas.append({
                    "Nodo": nodo,
                    "Actividad": actividad,
                    "Predecesor": pred,
                    "Tiempo de actividad": dur,
                    "Par ordenado": f"({par[0]},{par[1]})"
                })
    
    # Se determina el mayor "j" asignado para definir el nodo FIN
    max_j = max(ultimo_j.values())
    # Se asume que FIN depende del último nodo ingresado
    nodo_fin = {
        "Nodo": "FIN",
        "Actividad": "FIN",
        "Predecesor": ultimo_nodo if ultimo_nodo else "",
        "Tiempo de actividad": 0,
        "Par ordenado": f"({max_j},{max_j})"
    }
    filas.append(nodo_fin)
    
    # Se guarda la información en un archivo CSV
    nombre_archivo = "actividades.csv"
    with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo:
        campos = ["Nodo", "Actividad", "Predecesor", "Tiempo de actividad", "Par ordenado"]
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        for fila in filas:
            escritor.writerow(fila)
    
    print(f"\nLa información se ha guardado correctamente en el archivo '{nombre_archivo}'.")

if __name__ == "__main__":
    main()


Ingrese el número de actividades (nodos):  5



--- Ingresando datos para la actividad 1 ---


Ingrese el nombre del nodo (solo letras mayúsculas):  A
Ingrese el nombre de la actividad:  PRIMERA
Ingrese el número de nodos predecesores (0 significa que depende de INICIO):  0
Ingrese la duración de la actividad:  12



--- Ingresando datos para la actividad 2 ---


Ingrese el nombre del nodo (solo letras mayúsculas):  B
Ingrese el nombre de la actividad:  SEGUNDA
Ingrese el número de nodos predecesores (0 significa que depende de INICIO):  1
Ingrese el nombre del nodo predecesor:  A
Ingrese la duración de la actividad:  2


...


--- Ingresando datos para la actividad 4 ---


Ingrese el nombre del nodo (solo letras mayúsculas):  D
Ingrese el nombre de la actividad:  CUARTA
Ingrese el número de nodos predecesores (0 significa que depende de INICIO):  1
Ingrese el nombre del nodo predecesor:  B
Ingrese la duración de la actividad:  6



--- Ingresando datos para la actividad 5 ---


Ingrese el nombre del nodo (solo letras mayúsculas):  E
Ingrese el nombre de la actividad:  QUINTA
Ingrese el número de nodos predecesores (0 significa que depende de INICIO):  2
Ingrese el nombre del nodo predecesor 1:  C
Ingrese la duración de la actividad del predecesor C:  14
Ingrese el nombre del nodo predecesor 2:  D
Ingrese la duración de la actividad del predecesor D:  11



La información se ha guardado correctamente en el archivo 'actividades.csv'.
